In [1]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy import stats
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
import matplotlib.dates as mdates

def get_hstr(path):
    hstr = pd.read_csv(path)
    hstr['datetime'] = pd.to_datetime(hstr['CLCT_UNIX_TM'], unit='s', utc=True)
    hstr['datetime'] = hstr['datetime'].dt.tz_convert('Asia/Seoul').dt.strftime('%Y-%m-%d %H:%M:%S')
    hstr['datetime'] = hstr['datetime'].astype('datetime64[ns]')
    hstr.columns = ['현장교차로ID', '수집유닉스시각', '수집일시', '제어구분코드', '제어상태코드', '주기시간', '옵셋시간',
        'A링1현시시간', 'A링2현시시간', 'A링3현시시간', 'A링4현시시간', 'A링5현시시간', 'A링6현시시간',
        'A링7현시시간', 'A링8현시시간', 'B링1현시시간', 'B링2현시시간', 'B링3현시시간', 'B링4현시시간',
        'B링5현시시간', 'B링6현시시간', 'B링7현시시간', 'B링8현시시간', 'A링1현시보행시간',
        'A링2현시보행시간', 'A링3현시보행시간', 'A링4현시보행시간', 'A링5현시보행시간', 'A링6현시보행시간',
        'A링7현시보행시간', 'A링8현시보행시간', 'B링1현시보행시간', 'B링2현시보행시간', 'B링3현시보행시간',
        'B링4현시보행시간', 'B링5현시보행시간', 'B링6현시보행시간', 'B링7현시보행시간', 'B링8현시보행시간',
        '수집날짜시각']
    cols = list(hstr.columns).copy()
    cols.remove('수집날짜시각')
    cols.insert(1, '수집날짜시각')
    hstr = hstr[cols]
    return hstr

inter_nos = list(range(5031, 5048))

In [2]:
# 성남시 신호이력 데이터
s_tod_his = pd.read_csv('seongnam/S_TOD_HIS_1702597659892.csv')

# 인천시 신호이력 데이터
'''SELECT * FROM soitdspotintsoperhstr WHERE SPOT_INTS_ID BETWEEN 5031 AND 5047'''
csv_files = [f for f in os.listdir('incheon') if f.endswith('.csv')]

dfs = []
for file in csv_files:
    df_name = file.replace('.csv', '')
    print(df_name)
    globals()[df_name] = get_hstr(f'incheon/{file}')
    dfs.append(globals()[df_name])
    print(sorted([dt for dt in globals()[df_name]['수집날짜시각'] if dt.hour != 23])[0])
    print(sorted([dt for dt in globals()[df_name]['수집날짜시각'] if dt.hour != 23])[-1])
hstr = pd.concat(dfs).drop_duplicates().sort_values(by='수집날짜시각').reset_index(drop=True)
hstr = hstr[['현장교차로ID', '수집날짜시각', '제어구분코드', '제어상태코드', '주기시간', '옵셋시간',
             'A링1현시시간', 'A링2현시시간', 'A링3현시시간', 'A링4현시시간', 'A링5현시시간', 'A링6현시시간',
             'B링1현시시간', 'B링2현시시간', 'B링3현시시간', 'B링4현시시간', 'B링5현시시간', 'B링6현시시간']]
hstr = hstr[hstr.수집날짜시각.dt.day >= 12]
print(len(hstr))
print(sorted([dt for dt in hstr['수집날짜시각'] if dt.hour != 23])[0])
print(sorted([dt for dt in hstr['수집날짜시각'] if dt.hour != 23])[-1])

soitdspotintsoperhstr_202312110921
2023-12-10 09:00:00
2023-12-11 09:21:31
soitdspotintsoperhstr_202312120814
2023-12-11 09:00:57
2023-12-12 08:14:02
soitdspotintsoperhstr_202312131102
2023-12-12 09:00:58
2023-12-13 11:02:54
soitdspotintsoperhstr_202312141254
2023-12-13 09:00:58
2023-12-14 12:53:54
soitdspotintsoperhstr_202312150926
2023-12-14 09:00:58
2023-12-15 09:26:01
soitdspotintsoperhstr_202312151700
2023-12-15 09:00:58
2023-12-15 16:59:54
32466
2023-12-12 00:00:09
2023-12-15 16:59:54


In [3]:
infos = {}
# 신호이력
infos['hstr'] = {inter_no:hstr_temp for (inter_no, hstr_temp) in hstr.groupby('현장교차로ID')}
# 현시 개수
infos['number_of_phases'] = {}
for inter_no in inter_nos:
    a_ring_columns = [col for col in infos['hstr'][inter_no].columns if 'A링' in col]
    b_ring_columns = [col for col in infos['hstr'][inter_no].columns if 'B링' in col]
    max_a_ring = max([int(col[2]) for col in a_ring_columns if infos['hstr'][inter_no][col].any() != 0])
    max_b_ring = max([int(col[2]) for col in b_ring_columns if infos['hstr'][inter_no][col].any() != 0])
    if max_a_ring == max_b_ring:
        infos['number_of_phases'][inter_no] = max_a_ring
    else:
        raise "A링, B링 현시번호 최댓값이 서로 다름"
# 현시시간 목록
infos['durations'] = {inter_no:np.unique(infos['hstr'][inter_no].iloc[:,4:].values.flatten()) for inter_no in inter_nos}

In [4]:
# TOD 계획 데이터
todplans = pd.read_csv("soitdtodplan_202312181440.csv")
todplans.columns = ["현장교차로ID", "시각계획번호", "시각운영번호", "수집유닉스시각", "수집일시", "시작시", "시작분", "현시운영번호", "현시운영계획번호"]
todplans = todplans[(todplans['수집일시'] == 20231212) & (todplans['시각계획번호'] == 1) & (todplans['시작시']!=0)]

# 전이시각, ID
infos['transition_times'] = {inter_no:todplans[todplans.현장교차로ID == inter_no][['시작시', '시작분']] for inter_no in inter_nos}
infos['ID'] = {inter_no:todplans[todplans.현장교차로ID == inter_no].현시운영계획번호.unique() for inter_no in inter_nos}

# 시간계획 데이터
timeplans = pd.read_csv("soitdtimeplan_202312180943.csv")
timeplans.columns = ["현장교차로ID", "시간계획번호", "현시운영번호", "현시운영계획번호", "수집유닉스시각", "수집일시", "주기시간", "옵셋시간",
                     "A링1현시시간", "A링2현시시간", "A링3현시시간", "A링4현시시간", "A링5현시시간", "A링6현시시간", "A링7현시시간", "A링8현시시간",
                     "B링1현시시간", "B링2현시시간", "B링3현시시간", "B링4현시시간", "B링5현시시간", "B링6현시시간", "B링7현시시간", "B링8현시시간"]
timeplans = timeplans[(timeplans.시간계획번호 == 1)]
timeplan_list = []
for inter_no in inter_nos:
    timeplan_list.append(timeplans[(timeplans.현장교차로ID==inter_no) & (timeplans.현시운영계획번호.isin(infos['ID'][inter_no]))])
timeplans = pd.concat(timeplan_list)

# 이상치로 판단하지 않는 현시시간 / 이상치로 판단하는 현시시간
infos['accepted_durations'] = {}
infos['unaccepted_durations'] = {}

# 현시시간 빈도, 옵셋
infos['unique_durations'] = {}
infos['duration_frequencies'] = {}
infos['offsets'] = {}
infos['cycles'] = {}

for inter_no in inter_nos:
    phase_times = np.unique(timeplans[timeplans.현장교차로ID == inter_no].iloc[:, 8:].values.flatten())
    accepted_phase_times = np.unique(np.concatenate([phase_times - 1, phase_times, phase_times + 1]))
    infos['accepted_durations'][inter_no] = accepted_phase_times
    unaccepted_phase_times = sorted(set(infos['durations'][inter_no]) - set(accepted_phase_times))
    infos['unaccepted_durations'][inter_no] = unaccepted_phase_times

    unique_durations, frequencies = np.unique(infos['hstr'][inter_no].iloc[:,6:].values, return_counts=True)
    sorted_indices = np.argsort(frequencies)[::-1]
    infos['unique_durations'][inter_no] = unique_durations[sorted_indices]
    infos['duration_frequencies'][inter_no] = frequencies[sorted_indices]

    infos['offsets'][inter_no] = {}
    infos['cycles'][inter_no] = {}
    for _, row in infos['transition_times'][inter_no].iterrows():
        hour = row.시작시
        minute = row.시작분
        ID = todplans[(todplans.현장교차로ID==inter_no) & (todplans.시작시==hour) & (todplans.시작분==minute)].iloc[0].현시운영계획번호
        offset = timeplans[(timeplans.현장교차로ID==inter_no)&(timeplans.현시운영계획번호==ID)].iloc[0].옵셋시간
        cycle = timeplans[(timeplans.현장교차로ID==inter_no)&(timeplans.현시운영계획번호==ID)].iloc[0].주기시간
        infos['offsets'][inter_no][(hour, minute)] = offset
        infos['cycles'][inter_no][(hour, minute)] = cycle


In [5]:
k = 3 # context number

# 이상치 dataframe, 이상치를 포함한 주변에 대한 dataframe, 정상 dataframe
infos['outlier_dfs'] = {}
infos['context_dfs'] = {}
infos['context_dfs_list'] = {}
infos['normal_dfs'] = {}
hstr_context = []
hstr_outlier = []
for inter_no in inter_nos:
    df = infos['hstr'][inter_no].sort_values(by='수집날짜시각').reset_index(drop=True)
    outliers = infos['unaccepted_durations'][inter_no]
    ring_columns = [col for col in df.columns if '현시시간' in col]
    outlier_indices = df[df[ring_columns].isin(outliers).any(axis=1)].index
    context_indices = [] # 각 아웃라이어 인덱스에 대해 위아래 네 행의 인덱스를 구합니다.
    context_indices_list = []
    for idx in outlier_indices:
        start_idx = max(idx - k, 0)  # 데이터프레임의 시작을 넘어가지 않도록 합니다.
        end_idx = min(idx + k + 1, len(df))  # 데이터프레임의 끝을 넘어가지 않도록 합니다.
        context_indices.extend(range(start_idx, end_idx))
        context_indices_list.append(list(range(start_idx, end_idx)))
    context_indices = sorted(context_indices)
    normal_indices = sorted(set(df.index) - set(outlier_indices))

    df['전이시간여부'] = False
    for _, row in infos['transition_times'][inter_no].iterrows():
        hour = row.시작시
        minute = row.시작분
        transition_range = ((df['수집날짜시각'].dt.hour==hour) & (df['수집날짜시각'].dt.minute >= minute) & (df['수집날짜시각'].dt.minute <= minute+20))
        df.loc[transition_range, '전이시간여부'] = True

    df['이상치존재'] = False
    df.loc[outlier_indices, '이상치존재'] = True

    infos['outlier_dfs'][inter_no] = df.iloc[outlier_indices].drop_duplicates()
    infos['context_dfs'][inter_no] = df.iloc[context_indices].drop_duplicates()
    infos['context_dfs_list'][inter_no] = []
    for context_indices_ in context_indices_list:
        infos['context_dfs_list'][inter_no].append(df.iloc[context_indices_])
    infos['normal_dfs'][inter_no]  = df.iloc[normal_indices]
    hstr_context.append(infos['context_dfs'][inter_no])
    hstr_outlier.append(infos['outlier_dfs'][inter_no])
hstr_context = pd.concat(hstr_context)
hstr_outlier = pd.concat(hstr_outlier)

In [21]:
dfs = []
for inter_no in inter_nos:
    DF = infos['context_dfs'][inter_no]
    temp = []
    for i in DF.index:
        if not temp or i == temp[-1] + 1:
            temp.append(i)
        else:
            dfs.append(DF.loc[temp])
            temp = [i]
    if temp:
        dfs.append(DF.loc[temp])

In [22]:
with pd.option_context('display.max_columns', None):
    display(pd.read_csv('incheon/soitdspotintsoperhstr_202312120814.csv'))

,SPOT_INTS_ID,CLCT_UNIX_TM,CLCT_DT,CTRL_SE_CD,CTRL_STTS_CD,CYCL_HR,OFST_HR,A_RING_1_PHAS_HR,A_RING_2_PHAS_HR,A_RING_3_PHAS_HR,A_RING_4_PHAS_HR,A_RING_5_PHAS_HR,A_RING_6_PHAS_HR,A_RING_7_PHAS_HR,A_RING_8_PHAS_HR,B_RING_1_PHAS_HR,B_RING_2_PHAS_HR,B_RING_3_PHAS_HR,B_RING_4_PHAS_HR,B_RING_5_PHAS_HR,B_RING_6_PHAS_HR,B_RING_7_PHAS_HR,B_RING_8_PHAS_HR,A_RING_1_PHAS_AMBL_HR,A_RING_2_PHAS_AMBL_HR,A_RING_3_PHAS_AMBL_HR,A_RING_4_PHAS_AMBL_HR,A_RING_5_PHAS_AMBL_HR,A_RING_6_PHAS_AMBL_HR,A_RING_7_PHAS_AMBL_HR,A_RING_8_PHAS_AMBL_HR,B_RING_1_PHAS_AMBL_HR,B_RING_2_PHAS_AMBL_HR,B_RING_3_PHAS_AMBL_HR,B_RING_4_PHAS_AMBL_HR,B_RING_5_PHAS_AMBL_HR,B_RING_6_PHAS_AMBL_HR,B_RING_7_PHAS_AMBL_HR,B_RING_8_PHAS_AMBL_HR
0,5046,1702393195,20231212,3,1,100,95,20,80,0,0,0,0,0,0,20,80,0,0,0,0,0,0,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5036,1702336442,20231212,3,1,181,123,39,49,28,30,35,0,0,0,39,49,28,30,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,5040,1702336439,20231212,3,1,180,119,41,83,16,40,0,0,0,0,41,83,16,40,0,0,0,0,0,38,0,32,0,0,0,0,32,36,0,0,0,0,0,0
3,5044,1702336439,20231212,3,1,180,120,40,62,20,20,22,16,0,0,40,62,20,20,22,16,0,0,36,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5032,1702336435,20231212,3,1,170,55,24,61,24,37,24,0,0,0,24,61,24,37,24,0,0,0,0,0,0,35,0,0,0,0,0,0,0,35,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9104,5039,1702252906,20231211,3,1,181,106,45,136,0,0,0,0,0,0,45,136,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9105,5033,1702252902,20231211,3,1,170,32,42,86,42,0,0,0,0,0,42,86,42,0,0,0,0,0,36,46,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9106,5043,1702252898,20231211,3,1,180,98,42,79,8,11,40,0,0,0,42,79,8,11,40,0,0,0,0,32,0,5,35,0,0,0,34,0,0,0,0,0,0,0
9107,5038,1702252888,20231211,3,1,169,19,26,74,49,20,0,0,0,0,26,74,49,20,0,0,0,0,0,0,0,17,0,0,0,0,21,32,21,0,0,0,0,0


In [27]:
hstr.제어구분코드.value_counts()

제어구분코드
3    26873
0     5593
Name: count, dtype: int64

In [26]:
hstr.제어상태코드.value_counts()

제어상태코드
1    26118
0     5519
2      754
3       75
Name: count, dtype: int64

In [24]:
len(dfs)

591

In [7]:
import random
for _ in range(10):
    p = random.choice(range(len(dfs)))
    display(dfs[p])

,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
671,5039,2023-12-13 06:58:04,3,1,170,94,45,125,0,0,0,0,45,125,0,0,0,0,False,False
672,5039,2023-12-13 07:00:56,3,1,171,96,45,126,0,0,0,0,45,126,0,0,0,0,True,False
673,5039,2023-12-13 07:03:45,3,1,169,45,45,124,0,0,0,0,45,124,0,0,0,0,True,False
674,5039,2023-12-13 07:07:44,3,2,239,104,45,194,0,0,0,0,45,194,0,0,0,0,True,True
675,5039,2023-12-13 07:10:45,3,1,181,105,45,136,0,0,0,0,45,136,0,0,0,0,True,False
676,5039,2023-12-13 07:13:45,3,1,180,106,45,135,0,0,0,0,45,135,0,0,0,0,True,False
677,5039,2023-12-13 07:16:44,3,1,179,104,45,134,0,0,0,0,45,134,0,0,0,0,True,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
900,5047,2023-12-13 22:28:32,3,1,170,162,33,69,24,44,0,0,33,69,24,44,0,0,False,False
901,5047,2023-12-13 22:31:22,3,1,170,162,33,69,24,44,0,0,33,69,24,44,0,0,True,False
902,5047,2023-12-13 22:34:12,3,1,170,132,33,69,24,44,0,0,33,69,24,44,0,0,True,False
903,5047,2023-12-13 22:37:14,3,2,182,154,35,74,23,50,0,0,35,74,23,50,0,0,True,True
904,5047,2023-12-13 22:39:54,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,True,False
905,5047,2023-12-13 22:42:34,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,True,False
906,5047,2023-12-13 22:45:14,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,True,False
907,5047,2023-12-13 22:47:54,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,True,False
908,5047,2023-12-13 22:50:34,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,True,False
909,5047,2023-12-13 22:53:14,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,False,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
987,5042,2023-12-13 23:26:52,0,0,160,92,34,58,29,18,13,8,34,58,29,18,13,8,False,False
988,5042,2023-12-13 23:29:32,0,0,160,92,34,58,29,18,13,8,34,58,29,18,13,8,False,False
989,5042,2023-12-13 23:32:12,0,0,160,92,34,58,29,18,13,8,34,58,29,18,13,8,True,False
990,5042,2023-12-13 23:34:22,0,0,130,112,33,31,29,16,13,8,33,31,29,16,13,8,True,True
991,5042,2023-12-13 23:36:32,0,0,130,92,33,31,29,16,13,8,33,31,29,16,13,8,True,True
992,5042,2023-12-13 23:38:53,0,0,141,83,33,42,29,16,13,8,33,42,29,16,13,8,True,True
993,5042,2023-12-13 23:41:23,0,0,150,83,33,51,29,16,13,8,33,51,29,16,13,8,True,False
994,5042,2023-12-13 23:43:53,0,0,150,83,33,51,29,16,13,8,33,51,29,16,13,8,True,False
995,5042,2023-12-13 23:46:23,0,0,150,83,33,51,29,16,13,8,33,51,29,16,13,8,True,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
852,5044,2023-12-13 16:59:00,3,1,170,110,40,62,17,19,16,16,40,62,17,19,16,16,False,False
853,5044,2023-12-13 17:01:50,3,1,170,110,40,62,17,19,16,16,40,62,17,19,16,16,True,False
854,5044,2023-12-13 17:04:40,3,1,170,100,40,62,17,19,16,16,40,62,17,19,16,16,True,False
855,5044,2023-12-13 17:07:57,3,2,198,118,45,73,22,22,20,16,45,73,22,22,20,16,True,True
856,5044,2023-12-13 17:10:59,3,1,182,120,41,67,20,20,18,16,41,67,20,20,18,16,True,False
857,5044,2023-12-13 17:13:59,3,1,180,120,40,66,20,20,18,16,40,66,20,20,18,16,True,False
858,5044,2023-12-13 17:16:59,3,1,180,120,40,66,20,20,18,16,40,66,20,20,18,16,True,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
1954,5033,2023-12-15 16:52:18,3,1,170,48,42,86,42,0,0,0,42,86,42,0,0,0,False,False
1955,5033,2023-12-15 16:55:08,3,1,170,48,42,86,42,0,0,0,42,86,42,0,0,0,False,False
1956,5033,2023-12-15 16:57:58,3,1,170,48,42,86,42,0,0,0,42,86,42,0,0,0,False,False
1957,5033,2023-12-15 23:59:30,3,1,130,51,36,14,27,27,26,0,36,14,27,27,26,0,False,True


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
923,5041,2023-12-13 20:28:54,3,1,180,114,46,113,11,10,0,0,46,113,11,10,0,0,False,False
924,5041,2023-12-13 20:31:54,3,1,180,115,46,113,11,10,0,0,46,113,11,10,0,0,True,False
925,5041,2023-12-13 20:34:53,3,1,179,143,46,112,11,10,0,0,46,112,11,10,0,0,True,False
926,5041,2023-12-13 20:37:16,3,2,143,116,46,77,10,10,0,0,46,77,10,10,0,0,True,True
927,5041,2023-12-13 20:39:54,3,2,158,104,46,92,10,10,0,0,46,92,10,10,0,0,True,False
928,5041,2023-12-13 20:42:44,3,1,170,104,46,103,11,10,0,0,46,103,11,10,0,0,True,False
929,5041,2023-12-13 20:45:34,3,1,170,105,46,103,11,10,0,0,46,103,11,10,0,0,True,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
1559,5045,2023-12-15 05:56:54,0,0,180,174,27,81,36,36,0,0,27,81,36,36,0,0,False,False
1560,5045,2023-12-15 05:59:54,0,0,180,175,27,81,36,36,0,0,27,81,36,36,0,0,False,False
1561,5045,2023-12-15 06:02:54,0,0,180,175,27,81,36,36,0,0,27,81,36,36,0,0,True,False
1562,5045,2023-12-15 06:06:43,0,0,229,3,41,112,37,39,0,0,27,126,37,39,0,0,True,True
1563,5045,2023-12-15 06:10:23,0,0,220,3,38,108,36,38,0,0,25,121,36,38,0,0,True,False
1564,5045,2023-12-15 06:14:03,0,0,220,4,38,108,36,38,0,0,25,121,36,38,0,0,True,False
1565,5045,2023-12-15 06:17:43,0,0,220,3,38,108,36,38,0,0,25,121,36,38,0,0,True,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
437,5034,2023-12-12 20:56:37,3,1,180,158,57,33,45,45,0,0,31,59,45,45,0,0,False,False
438,5034,2023-12-12 20:59:38,3,1,180,158,57,33,45,45,0,0,31,59,45,45,0,0,False,False
439,5034,2023-12-12 21:02:38,3,1,180,108,57,33,45,45,0,0,31,59,45,45,0,0,True,False
440,5034,2023-12-12 21:06:17,3,2,219,157,65,39,58,57,0,0,37,67,58,57,0,0,True,True
441,5034,2023-12-12 21:09:07,3,1,170,157,51,30,45,44,0,0,29,52,45,44,0,0,True,False
442,5034,2023-12-12 21:11:57,3,1,170,157,51,30,45,44,0,0,29,52,45,44,0,0,True,False
443,5034,2023-12-12 21:14:46,3,1,170,157,51,30,45,44,0,0,29,52,45,44,0,0,True,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
628,5034,2023-12-13 05:09:46,3,1,150,135,43,21,43,43,0,0,21,43,43,43,0,0,False,False
629,5034,2023-12-13 05:12:16,3,1,150,135,43,21,43,43,0,0,21,43,43,43,0,0,False,False
630,5034,2023-12-13 05:14:46,3,1,150,135,43,21,43,43,0,0,21,43,43,43,0,0,False,False
631,5034,2023-12-13 05:16:54,3,1,105,135,43,21,41,0,0,0,21,43,41,0,0,0,False,True
632,5034,2023-12-13 05:17:38,0,0,67,8,0,0,0,67,0,0,0,0,0,67,0,0,False,True
633,5034,2023-12-13 05:20:08,0,0,150,7,43,21,43,43,0,0,21,43,43,43,0,0,False,False
634,5034,2023-12-13 05:22:36,3,2,150,7,43,21,43,43,0,0,21,43,43,43,0,0,False,False
635,5034,2023-12-13 05:25:07,3,2,150,7,43,21,43,43,0,0,21,43,43,43,0,0,False,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
1508,5046,2023-12-15 03:10:09,0,0,150,9,24,42,42,42,0,0,24,42,42,42,0,0,False,False
1509,5046,2023-12-15 03:12:39,0,0,150,9,24,42,42,42,0,0,24,42,42,42,0,0,False,False
1510,5046,2023-12-15 03:15:10,0,0,151,9,24,42,43,42,0,0,24,42,43,42,0,0,False,False
1511,5046,2023-12-15 03:17:38,0,0,148,7,22,41,42,43,0,0,22,41,42,43,0,0,False,True
1512,5046,2023-12-15 03:20:08,0,0,150,7,24,42,42,42,0,0,24,42,42,42,0,0,False,False
1513,5046,2023-12-15 03:22:38,0,0,150,8,24,42,42,42,0,0,24,42,42,42,0,0,False,False
1514,5046,2023-12-15 03:25:08,0,0,150,8,24,42,42,42,0,0,24,42,42,42,0,0,False,False


In [8]:
"""
5. 전이시간 바깥에서 발생하는 이상현시시간 관찰

이상치존재가 True이고 전이시간여부가 False인 경우에 대하여 앞의 두 번 뒤의 두 번을 포함해 관찰합니다.
"""

infos['context_dfs_outside_the_transition_ranges'] = {}
infos['outlier_lists'] = {}
infos['outliers'] = []
for inter_no in inter_nos:
    dfs = infos['context_dfs_list'][inter_no]
    dfs = [dfs[i] for i in range(len(dfs)) if ~dfs[i].iloc[2]['전이시간여부']]
    infos['context_dfs_outside_the_transition_ranges'][inter_no] = pd.concat(dfs).drop_duplicates()

    df = infos['outlier_dfs'][inter_no].iloc[:,6:-2]
    is_outlier = df.isin(infos['unaccepted_durations'][inter_no])
    temp = df[is_outlier].values.flatten()
    temp = np.unique(temp[pd.notna(temp)])
    infos['outlier_lists'][inter_no] = np.unique(temp[pd.notna(temp)])
    infos['outliers'].extend(infos['outlier_lists'][inter_no])

In [9]:
def display_context_dfs_outside_the_transition_ranges(inter_no):
    DF = infos['context_dfs_outside_the_transition_ranges'][inter_no]
    dfs = []
    temp = []
    for i in DF.index:
        if not temp or i == temp[-1] + 1:
            temp.append(i)
        else:
            dfs.append(DF.loc[temp])
            temp = [i]
    if temp:
        dfs.append(DF.loc[temp])
    for df in dfs:
        display(df)
    # return dfs

In [10]:
for inter_no in inter_nos:
    display_context_dfs_outside_the_transition_ranges(inter_no)

,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
775,5031,2023-12-13 10:29:00,3,1,160,140,49,20,27,29,35,0,49,20,27,29,35,0,False,False
776,5031,2023-12-13 10:31:40,3,1,160,140,49,20,27,29,35,0,49,20,27,29,35,0,False,False
777,5031,2023-12-13 10:34:20,3,1,160,140,49,20,27,29,35,0,49,20,27,29,35,0,False,False
778,5031,2023-12-13 10:39:07,3,1,31,107,49,20,27,156,35,0,49,20,27,156,35,0,False,True
779,5031,2023-12-13 10:41:27,0,0,140,87,37,16,27,25,35,0,37,16,27,25,35,0,False,False
780,5031,2023-12-13 10:45:00,3,2,213,140,69,31,31,43,39,0,69,31,31,43,39,0,False,True
781,5031,2023-12-13 10:47:40,3,1,160,140,49,20,27,29,35,0,49,20,27,29,35,0,False,False
782,5031,2023-12-13 10:50:20,3,1,160,140,49,20,27,29,35,0,49,20,27,29,35,0,False,False
783,5031,2023-12-13 10:53:00,3,1,160,140,49,20,27,29,35,0,49,20,27,29,35,0,False,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
909,5031,2023-12-13 17:59:38,3,1,180,159,65,20,27,33,35,0,65,20,27,33,35,0,False,False
910,5031,2023-12-13 18:02:38,3,1,180,159,65,20,27,33,35,0,65,20,27,33,35,0,False,False
911,5031,2023-12-13 18:05:38,3,1,180,159,65,20,27,33,35,0,65,20,27,33,35,0,False,False
912,5031,2023-12-13 18:10:47,3,1,52,107,65,20,27,161,35,0,65,20,27,161,35,0,False,True
913,5031,2023-12-13 18:13:07,0,0,140,67,37,16,27,25,35,0,37,16,27,25,35,0,False,False
914,5031,2023-12-13 18:17:06,3,2,239,126,90,31,31,48,39,0,90,31,31,48,39,0,False,True
915,5031,2023-12-13 18:20:39,3,2,213,159,78,25,31,40,39,0,78,25,31,40,39,0,False,True
916,5031,2023-12-13 18:23:39,3,1,180,160,65,20,27,33,35,0,65,20,27,33,35,0,False,False
917,5031,2023-12-13 18:26:38,3,1,179,159,65,20,27,32,35,0,65,20,27,32,35,0,False,False
918,5031,2023-12-13 18:29:38,3,1,180,158,65,20,27,33,35,0,65,20,27,33,35,0,False,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
1763,5031,2023-12-15 07:21:58,0,0,1,58,126,35,27,34,35,0,126,35,27,34,35,0,False,True
1764,5031,2023-12-15 07:24:18,0,0,140,19,37,16,27,25,35,0,37,16,27,25,35,0,False,False
1765,5031,2023-12-15 07:26:49,3,2,151,169,39,25,27,25,35,0,39,25,27,25,35,0,False,False
1766,5031,2023-12-15 07:29:41,3,2,172,161,47,32,27,31,35,0,47,32,27,31,35,0,False,True
1767,5031,2023-12-15 07:32:41,3,1,180,161,49,35,27,34,35,0,49,35,27,34,35,0,False,False
1768,5031,2023-12-15 07:35:41,3,1,180,161,49,35,27,34,35,0,49,35,27,34,35,0,False,False
1769,5031,2023-12-15 07:38:41,3,1,180,161,49,35,27,34,35,0,49,35,27,34,35,0,False,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
0,5032,2023-12-12 00:00:40,3,1,140,40,20,43,20,37,20,0,20,43,20,37,20,0,False,False
1,5032,2023-12-12 00:03:20,3,2,160,60,25,51,24,37,23,0,25,51,24,37,23,0,False,True
2,5032,2023-12-12 00:05:40,3,1,140,60,20,43,20,37,20,0,20,43,20,37,20,0,False,False
3,5032,2023-12-12 00:08:00,3,1,140,60,20,43,20,37,20,0,20,43,20,37,20,0,False,False
4,5032,2023-12-12 00:10:20,3,1,140,60,20,43,20,37,20,0,20,43,20,37,20,0,False,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
189,5032,2023-12-12 07:34:15,3,1,170,55,24,61,24,37,24,0,24,61,24,37,24,0,False,False
190,5032,2023-12-12 07:37:05,3,1,170,55,24,61,24,37,24,0,24,61,24,37,24,0,False,False
191,5032,2023-12-12 07:39:55,3,1,170,55,24,61,24,37,24,0,24,61,24,37,24,0,False,False
192,5032,2023-12-12 07:42:27,3,1,133,37,0,55,21,37,20,0,0,55,21,37,20,0,False,True
193,5032,2023-12-12 07:44:24,0,0,117,154,18,24,19,37,19,0,18,24,19,37,19,0,False,True
194,5032,2023-12-12 07:48:10,3,2,226,40,35,84,35,37,35,0,35,84,35,37,35,0,False,True
195,5032,2023-12-12 07:51:15,3,2,185,55,27,67,27,37,27,0,27,67,27,37,27,0,False,False
196,5032,2023-12-12 07:54:05,3,1,170,55,24,61,24,37,24,0,24,61,24,37,24,0,False,False
197,5032,2023-12-12 07:56:55,3,1,170,55,24,61,24,37,24,0,24,61,24,37,24,0,False,False


...

,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
907,5047,2023-12-13 22:47:54,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,True,False
908,5047,2023-12-13 22:50:34,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,True,False
909,5047,2023-12-13 22:53:14,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,False,False
910,5047,2023-12-13 22:56:53,3,1,220,53,31,125,20,44,0,0,31,125,20,44,0,0,False,True
911,5047,2023-12-13 22:58:59,0,0,126,19,25,37,20,44,0,0,25,37,20,44,0,0,False,True
912,5047,2023-12-13 23:01:14,3,2,135,154,25,46,20,44,0,0,25,46,20,44,0,0,False,True
913,5047,2023-12-13 23:03:54,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,False,False
914,5047,2023-12-13 23:06:34,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,False,False
915,5047,2023-12-13 23:09:14,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,False,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
1190,5047,2023-12-14 12:23:50,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False
1191,5047,2023-12-14 12:26:50,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False
1192,5047,2023-12-14 12:30:00,3,1,145,0,0,75,26,44,0,0,0,75,26,44,0,0,False,False
1193,5047,2023-12-14 12:32:50,0,0,170,170,33,71,22,44,0,0,33,71,22,44,0,0,False,True
1194,5047,2023-12-14 12:35:50,3,1,180,169,35,75,26,44,0,0,35,75,26,44,0,0,False,False
1195,5047,2023-12-14 12:38:51,3,1,181,170,35,76,26,44,0,0,35,76,26,44,0,0,False,False
1196,5047,2023-12-14 12:41:50,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
1261,5047,2023-12-14 15:56:50,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False
1262,5047,2023-12-14 15:59:51,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False
1263,5047,2023-12-14 16:02:50,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False
1264,5047,2023-12-14 16:08:46,3,1,99,165,35,75,26,219,0,0,35,75,26,219,0,0,False,True
1265,5047,2023-12-14 16:10:52,0,0,126,111,25,37,20,44,0,0,25,37,20,44,0,0,False,True
1266,5047,2023-12-14 16:14:51,3,2,239,170,46,100,35,58,0,0,46,100,35,58,0,0,False,True
1267,5047,2023-12-14 16:17:51,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False
1268,5047,2023-12-14 16:20:51,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False
1269,5047,2023-12-14 16:23:51,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
1396,5047,2023-12-14 22:53:14,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,False,False
1397,5047,2023-12-14 22:55:54,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,False,False
1398,5047,2023-12-14 22:58:59,0,0,185,19,56,65,20,44,0,0,56,65,20,44,0,0,False,False
1399,5047,2023-12-14 23:01:14,0,0,135,154,25,46,20,44,0,0,25,46,20,44,0,0,False,True
1400,5047,2023-12-14 23:03:54,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,False,False
1401,5047,2023-12-14 23:06:35,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,False,False
1402,5047,2023-12-14 23:09:14,3,1,160,154,31,65,20,44,0,0,31,65,20,44,0,0,False,False


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,전이시간여부,이상치존재
1684,5047,2023-12-15 12:41:50,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False
1685,5047,2023-12-15 12:44:50,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False
1686,5047,2023-12-15 12:49:13,3,1,70,72,0,0,26,44,0,0,0,0,26,44,0,0,False,False
1687,5047,2023-12-15 12:51:19,0,0,126,18,25,37,20,44,0,0,25,37,20,44,0,0,False,True
1688,5047,2023-12-15 12:53:51,3,2,152,170,28,60,20,44,0,0,28,60,20,44,0,0,False,True
1689,5047,2023-12-15 12:56:50,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False
1690,5047,2023-12-15 12:59:51,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False
1691,5047,2023-12-15 13:02:50,3,1,180,170,35,75,26,44,0,0,35,75,26,44,0,0,False,False


In [11]:
# context = []
# for i in (hstr.제어상태코드==2).index:
#     start_idx = max(idx - k, 0)  # 데이터프레임의 시작을 넘어가지 않도록 합니다.
#     end_idx = min(idx + k + 1, len(hstr))  # 데이터프레임의 끝을 넘어가지 않도록 합니다.
#     context_indices.extend(range(start_idx, end_idx))
#     context.extend(list(range(i-k,i+k)))
# hstr[hstr.index.isin(context)]